# ĐỒ ÁN THỰC HÀNH
# CSC17104 – LẬP TRÌNH CHO KHOA HỌC DỮ LIỆU

Giảng viên: Bùi Tiến Lên

Thông tin thành viên:
1. Nguyễn Văn Tuấn Đạt 19120472
2. Phan Xuân Hoài 20120481
3. 
4. 

## Import

In [1]:
import numpy as np
import pandas as pd

## A. Thu thập dữ liệu
- Tên dữ liệu: HR Analytics: Job Change of Data Scientists
- Nguồn lấy dữ liệu: kaggle
- Link dữ liệu: https://www.kaggle.com/datasets/arashnic/hr-analytics-job-change-of-data-scientists
- License: CC0: Public Domain
- Mô tả về dữ liệu: dữ liệu gồm tập train và tập test. Trong đồ án này, nhóm sẽ sử dụng tập train.
            + Dữ liệu gồm 14 cột và 19158 dòng.

In [2]:
df = pd.read_csv('aug_train.csv', encoding='utf-8')
df.shape

(19158, 14)

## B. Khám phá dữ liệu (thường đan xen với pha tiền xử lý dữ liệu)

### **1.Mỗi dòng có ý nghĩa gì? Có vấn đề với các dòng có ý nghĩa khác nhau không?**

* Mỗi dòng là thông tin của từng ứng viên, trong đó mỗi ô của dòng là dữ liệu tương ứng với cột của ô đó
* Không có vấn đề các dòng có ý nghĩa khác nhau

### 2.Mỗi cột có ý nghĩa gì?
            
            - enrollee_id : ID với từng ứng viên

            - city: Mã thành phố

            - city_ development _index : Chỉ số phát triển của thành phố (theo tỷ lệ)

            - gender: Giới tính

            - relevent_experience: Kinh nghiệm liên quan của ứng viên

            - enrolled_university: Loại khóa học đại học đã đăng ký nếu có

            - education_level: Trình độ học vấn

            - major_discipline: Ngành học chính

            - experience: Kinh nghiệm (tính theo năm)

            - company_size: Số lượng nhân viên trong công ty của người sử dụng lao động hiện tại

            - company_type : Loại chủ lao động hiện tại

            - lastnewjob: Sự khác biệt về số năm giữa công việc trước đây và công việc hiện tại

            - training_hours: Thời gian hoàn thành huấn luyện

            - target: 0 – Không nhảy việc, 1 – Nhảy việc

### 3.Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp hay không?

In [3]:
df.dtypes

enrollee_id                 int64
city                       object
city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
target                    float64
dtype: object

In [4]:
df['experience'].replace({'1':'1-5',
                                '2':'1-5',
                                '3':'1-5',
                                '4':'1-5',
                                '5':'1-5',
                                '6':'6-10',
                                '7':'6-10',
                                '8':'6-10',
                                '9':'6-10',
                                '10':'6-10',
                                '11':'11-15',
                                '12':'11-15',
                                '13':'11-15',
                                '14':'11-15',
                                '15':'11-15',
                                '16':'16-20',
                                '17':'16-20',
                                '18':'16-20',
                                '19':'16-20',
                                '20':'16-20'},inplace=True)

### 4.Với mỗi cột, các giá trị được phân bố như thế nào?

#### 4.1 Với các cột có giá trị dạng số

**Các cột dạng số:** _'enrollee_id', 'city_development_index', 'training_hours', 'target'_ 

DataFrame `summary_df` bên dưới chứa thông tin phân bố giá trị của các cột trên:
- Tên của các cột là tên của các cột số trong `df`
- Tên của các dòng có ý nghĩa là: 
 + `"missing_ratio"`: tỉ lệ phần trăm các giá trị thiếu
 + `"count"`: số lượng giá trị 
 + `"mean"`: giá trị trung bình
 + `"std"`: độ lệch chuẩn
 + `"min"`: giá trị nhỏ nhất
 + `"25%"`: giá trị phân vị 25%
 + `"50%"`: giá trị phân vị 50%
 + `"75%"`: giá trị phân vị 75%
 + `"max"`: giá trị lớn nhất

In [5]:
summary_df = df.describe().round(1)
missing_ratio = pd.DataFrame(df[['enrollee_id', 'city_development_index', 'training_hours', 'target']].isna().sum() * 100/ len(df)).T.round(1)
missing_ratio = missing_ratio.set_axis(['missing_ratio'])
summary_df = pd.concat([missing_ratio, summary_df])
summary_df

,enrollee_id,city_development_index,training_hours,target
missing_ratio,0.0,0.0,0.0,0.0
count,19158.0,19158.0,19158.0,19158.0
mean,16875.4,0.8,65.4,0.2
std,9616.3,0.1,60.1,0.4
min,1.0,0.4,1.0,0.0
25%,8554.2,0.7,23.0,0.0
50%,16982.5,0.9,47.0,0.0
75%,25169.8,0.9,88.0,0.0
max,33380.0,0.9,336.0,1.0


#### 4.2 Với các cột có kiểu dữ liệu dạng phân loại (categorical)

**Các cột có kiểu dữ liệu dạng phân loại là:** _'city', 'gender', 'relevent_experience', 'enrolled_university',
       'education_level', 'major_discipline', 'experience', 'company_size',
       'company_type', 'last_new_job'_
       
**DataFrame `cate_col_df` bên dưới chứa các thông tin về phân bố giá trị của các cột trên:**
- Tên dòng(index) là các tên của các cột có kiểu dữ liệu phân loại trong `df`
- Với mỗi cột:
 + `'Missing Ratio'`: Phần trăm các giá trị thiếu
 + `'Count'`: Số lượng các giá trị khác nhau không tính giá trị thiếu
 + `'Values'`: Danh sách các giá trị khác nhau không chứa giá trị thiếu

In [56]:
# Lọc các cột có kiểu dữ liệu dạng phân loại
mask = df.dtypes == object
# Tạo dataFrame missing_df chứa phần trăm giá trị thiếu của các cột
missing_ar = np.mean(df.loc[:,mask].isna())
missing_df = pd.DataFrame(data= missing_ar, columns=['Missing Ratio']).round(3)

cate_col_dict = {}
for col_name in df.columns[mask]:
    cate_col_dict.setdefault(col_name, [])
    
    # Lấy ra các giá trị khác nhau không chứa giá trị nan và tính số lượng
    unique_val = df.loc[:,col_name].dropna().unique()
    unique_size = unique_val.size
    
    #Thêm các gía trị trên vào key tương ứng trong list
    cate_col_dict[col_name].append(unique_size)
    cate_col_dict[col_name].append(unique_val)
    
cate_col_df = pd.DataFrame(cate_col_dict, index=['Count', 'Values']).T
cate_col_df = pd.concat([missing_df, cate_col_df], axis = 1)
cate_col_df

,Missing Ratio,Count,Values
city,0.000,123,"[city_103, city_40, city_21, city_115, city_16..."
gender,0.235,3,"[Male, Female, Other]"
relevent_experience,0.000,2,"[Has relevent experience, No relevent experience]"
enrolled_university,0.020,3,"[no_enrollment, Full time course, Part time co..."
education_level,0.024,5,"[Graduate, Masters, High School, Phd, Primary ..."
major_discipline,0.147,6,"[STEM, Business Degree, Arts, Humanities, No M..."
experience,0.003,6,"[>20, 11-15, 1-5, <1, 6-10, 16-20]"
company_size,0.310,8,"[50-99, <10, 10000+, 5000-9999, 1000-4999, 10/..."
company_type,0.320,6,"[Pvt Ltd, Funded Startup, Early Stage Startup,..."
last_new_job,0.022,6,"[1, >4, never, 4, 3, 2]"


## C. Đưa ra các câu hỏi có ý nghĩa cần trả lời

## D. Tiền xử lý và phân tích dữ liệu để trả lời cho từng câu hỏi

## E. Tổng hợp lại quá trình thực hiện đồ án